In [1]:
import pandas
import tensorflow

print(pandas.__version__)
print(tensorflow.__version__)

1.3.3
2.6.0


In [2]:
import datetime as dt
from pathlib import Path
import os
import time
from datetime import datetime
from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [11]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [12]:
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

data = cleanse_recursive(data, shortest=2, least_click=5)

In [13]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    cutoff_time = final_time - dt.timedelta(n_days)
    
    train = data[data['Time'] < cutoff_time]
    test = data[data['Time'] >= cutoff_time]
    return train, test

tr, test = split_by_date(data, n_days=1)
tr, val = split_by_date(tr, n_days=1)

In [14]:
# Train 데이터의 아이템을 기준으로 인덱스 사전 생성
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    # Train 데이터에 없는 아이템은 -1로 처리
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [15]:
class SessionDataset:
    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())

    def get_click_offsets(self):
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

class FullSequenceDataLoader:
    def __init__(self, dataset: SessionDataset, batch_size=16):
        self.dataset = dataset
        self.batch_size = batch_size
        self.sessions = self._build_sessions()

    def _build_sessions(self):
        sessions = []
        for start, end in zip(self.dataset.click_offsets[:-1], self.dataset.click_offsets[1:]):
            session = self.dataset.df['item_idx'].values[start:end]
            if len(session) >= 2:
                sessions.append(session)
        return sessions

    # 'for' 루프를 가능하게 하는 __iter__ 메서드
    def __iter__(self):
        # 세션 리스트를 배치 사이즈만큼 순회
        for i in range(0, len(self.sessions), self.batch_size):
            batch = self.sessions[i:i+self.batch_size]
            input_batch = []
            target_batch = []

            for session in batch:
                input_batch.append(session[:-1])    # 예: [A, B, C]
                target_batch.append(session[-1])    # 예: D
            
            # 처리된 배치 데이터를 반환
            yield input_batch, target_batch

# 데이터 로더 생성
train_dataset = SessionDataset(tr)
train_loader = FullSequenceDataLoader(train_dataset, batch_size=16)

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

def create_model(args):
    inputs = Input(shape=(None, args.num_items))  # (배치 크기, 시퀀스 길이, 전체 아이템 수)
    x = GRU(args.hsz, return_sequences=False)(inputs)
    x = Dropout(args.drop_rate)(x)
    predictions = Dense(args.num_items, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

# 하이퍼파라미터 설정
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=16, hsz=50, drop_rate=0.1, lr=0.001, epochs=3, k=20)

model = create_model(args)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 3416)]      0         
_________________________________________________________________
gru_1 (GRU)                  (None, 50)                520200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3416)              174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


In [21]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = FullSequenceDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        tr_loader = tqdm(train_loader, desc=f'Train Epoch {epoch}')

        for input_seqs, target_items in tr_loader:
            # 1. 시퀀스 길이를 맞추기 위해 패딩(padding)을 적용
            input_seqs_padded = pad_sequences(input_seqs, padding='pre', maxlen=None)
            
            # 2. 패딩된 시퀀스를 One-hot-encoding
            input_ohe = to_categorical(input_seqs_padded, num_classes=args.num_items)
            
            # 3. target 아이템을 One-hot-encoding
            target_ohe = to_categorical(target_items, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy=result[1])
        
        # 각 에포크 종료 후 검증 데이터로 성능 평가
        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)
        print(f"\n\t - Recall@{args.k} epoch {epoch}: {val_recall:.4f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:.4f}\n")
        
def get_metrics(data, model, args, k: int):
    dataset = SessionDataset(data)
    loader = FullSequenceDataLoader(dataset, batch_size=args.batch_size)

    recall_list, mrr_list = [], []

    for input_seqs, label in tqdm(loader, desc='Evaluation'):
        input_seqs_padded = pad_sequences(input_seqs, padding='pre', maxlen=100)
        input_ohe = to_categorical(input_seqs_padded, num_classes=args.num_items)  # (B, T, num_items)

        pred = model.predict(input_ohe, batch_size=args.batch_size)  # (B, num_items)
        pred_arg = tf.argsort(pred, direction='DESCENDING')

        for i in range(len(label)):
            recall_list.append(recall_k(pred_arg[i], label[i], k))
            mrr_list.append(mrr_k(pred_arg[i], label[i], k))

    return np.mean(recall_list), np.mean(mrr_list)

def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0

# 모델 학습 시작
train_model(model, args)

Train Epoch 1: 378it [03:37,  1.74it/s, accuracy=0, train_loss=6.49]     
Evaluation: 1it [00:00, 11.60it/s]



	 - Recall@20 epoch 1: 0.0000
	 - MRR@20    epoch 1: 0.0000



Train Epoch 2: 378it [03:37,  1.74it/s, accuracy=0, train_loss=6.13]     
Evaluation: 1it [00:00, 11.89it/s]



	 - Recall@20 epoch 2: 0.0000
	 - MRR@20    epoch 2: 0.0000



Train Epoch 3: 378it [03:37,  1.74it/s, accuracy=0.125, train_loss=5.97] 
Evaluation: 1it [00:00, 14.09it/s]


	 - Recall@20 epoch 3: 0.0000
	 - MRR@20    epoch 3: 0.0000



In [23]:
# 1. 실험할 하이퍼파라미터 조합을 리스트로 정의
experiment_configs = [
    {'hsz': 50, 'drop_rate': 0.1, 'lr': 0.001, 'batch_size': 16},  # 베이스라인 실험
    {'hsz': 100, 'drop_rate': 0.1, 'lr': 0.001, 'batch_size': 16}, # 모델 복잡도 증가 (hsz)
    {'hsz': 50, 'drop_rate': 0.1, 'lr': 0.0005, 'batch_size': 32}, # 학습률 감소
]

# 각 실험의 최종 결과를 저장할 리스트
results = []

# 2. for 문을 통해 실험 자동화
for i, config in enumerate(experiment_configs):
    print(f"========== EXPERIMENT {i+1} START ==========")
    print(f"Config: {config}")

    # 현재 설정에 맞는 Args 인스턴스 생성
    args = Args(tr, val, test,
                batch_size=config['batch_size'],
                hsz=config['hsz'],
                drop_rate=config['drop_rate'],
                lr=config['lr'],
                epochs=5,  # 실험을 위해 에포크는 5회로 고정
                k=20)

    # 매 실험마다 새로운 모델 생성
    model = create_model(args)

    # 모델 학습
    train_model(model, args)

    # 학습 완료 후 테스트셋으로 최종 평가
    print(f"--- Final Test Evaluation for EXPERIMENT {i+1} ---")
    test_recall, test_mrr = get_metrics(args.test, model, args, args.k)
    print(f"Test Recall@20: {test_recall:.4f}")
    print(f"Test MRR@20: {test_mrr:.4f}")

    # 결과 저장
    config_result = config.copy()
    config_result['recall@20'] = test_recall
    config_result['mrr@20'] = test_mrr
    results.append(config_result)
    
    print(f"========== EXPERIMENT {i+1} END ==========\n\n")


# 3. 전체 실험 결과 비교
print("========== FINAL EXPERIMENT SUMMARY ==========")
results_df = pd.DataFrame(results)
display(results_df)

========== EXPERIMENT 1 START ==========
Config: {'hsz': 50, 'drop_rate': 0.1, 'lr': 0.001, 'batch_size': 16}
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 3416)]      0         
_________________________________________________________________
gru_3 (GRU)                  (None, 50)                520200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3416)              174216    
Total params: 694,416
Trainable params: 694,416
Non-trainable params: 0
_________________________________________________________________


Train Epoch 1: 378it [03:39,  1.72it/s, accuracy=0, train_loss=7.47]     
Evaluation: 1it [00:00,  3.59it/s]



	 - Recall@20 epoch 1: 0.0000
	 - MRR@20    epoch 1: 0.0000



Train Epoch 2: 378it [03:37,  1.74it/s, accuracy=0, train_loss=7.04]     
Evaluation: 1it [00:00, 13.20it/s]



	 - Recall@20 epoch 2: 0.0000
	 - MRR@20    epoch 2: 0.0000



...

Train Epoch 5: 189it [04:20,  1.38s/it, accuracy=0, train_loss=7.01]     
Evaluation: 1it [00:00, 14.82it/s]



	 - Recall@20 epoch 5: 0.0000
	 - MRR@20    epoch 5: 0.0000

--- Final Test Evaluation for EXPERIMENT 3 ---


Evaluation: 1it [00:00, 18.90it/s]

Test Recall@20: 0.0000
Test MRR@20: 0.0000
========== EXPERIMENT 3 END ==========


========== FINAL EXPERIMENT SUMMARY ==========


,hsz,drop_rate,lr,batch_size,recall@20,mrr@20
0,50,0.1,0.0010,16,0.0,0.0
1,100,0.1,0.0010,16,0.0,0.0
2,50,0.1,0.0005,32,0.0,0.0
